In [16]:
from PIL import Image, ImageDraw, ImageFont
import os
import piexif

def add_watermark(image_path, text, save_as=None):
    # Open the original image
    original = Image.open(image_path)
    
    # Make the image editable
    txt = Image.new('RGBA', original.size, (255,255,255,0))
    draw = ImageDraw.Draw(txt)

    # Choose a font and size for the watermark
    font = ImageFont.truetype("arial.ttf", 40)

    # Position for the watermark
    text_position = (10, 10)  # You can change this as needed

    # Apply the watermark
    draw.text(text_position, text, font=font, fill=(255,255,255,128))

    # Combine the watermark with the original image
    watermarked = Image.alpha_composite(original.convert('RGBA'), txt)

    # Convert the image to RGB mode before saving
    watermarked = watermarked.convert('RGB')

    # Save the image
    if save_as:
        watermarked.save(save_as, 'JPEG')
    else:
        watermarked.save(image_path, 'JPEG')


def get_camera_info(exif_data):
    if '0th' in exif_data:
        # Decode the bytes to a string if they exist
        make = exif_data['0th'].get(piexif.ImageIFD.Make, b'Unknown').decode('utf-8')
        model = exif_data['0th'].get(piexif.ImageIFD.Model, b'Unknown').decode('utf-8')
        camera_info = make + ' ' + model
    else:
        camera_info = 'Unknown'
    return camera_info



# Main function to process the images
def process_images(folder_path):
    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            # Extract EXIF data
            exif_data = piexif.load(image_path)
            # Get camera info from EXIF
            camera_info = get_camera_info(exif_data)
            # Create watermark text with camera info
            watermark_text = f"Camera: {camera_info}"
            # Add watermark to the image
            add_watermark(image_path, watermark_text, save_as=f"{image_path}_watermarked.jpg")

# Replace 'path_to_your_folder' with the path to the folder containing your images
process_images('../images/no_watermark')
